This notebook will be mainly used for the capstone project


In [3]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<b> Week 3 Assignment </b>

<b> 1.In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.</b>

<b> 2.Use the Notebook to build the code to scrape the following Wikipedia page</b>

In [61]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colo

In [62]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [63]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [64]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [68]:
# Drop any row which contains 'Not assigned' value
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]
df.head()

,PostalCode,Borough,Neighborhood
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn
5,M1H\n,Scarborough\n,Cedarbrae


In [49]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [50]:
df2[df2.Borough == 'East York']

,PostalCode,Borough,Neighborhood
35,M4B,East York,"Parkview Hill, Woodbine Gardens"
36,M4C,East York,Woodbine Heights
38,M4G,East York,Leaside
39,M4H,East York,Thorncliffe Park
40,M4J,East York,"East Toronto, Broadview North (Old East York)"


In [52]:
df2.shape

(103, 3)

<b> Use the Geocoder package or the csv file to create the following dataframe </b>

In [53]:
import pandas as pd

In [54]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 98 kB 6.1 MB/s eta 0:00:011


In [56]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')


[43.709020000000066, -79.36348999999996]

In [59]:
postal_codes = df['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]





In [60]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.72327,-79.45042
4,M4A,North York,Victoria Village,43.66253,-79.39188
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.66263,-79.52831
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.81139,-79.19662
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.74923,-79.36186


<b> Get the latitude and the longitude coordinates of each neighborhood obtained </b>

In [27]:
import geopy
from  geopy.geocoders import Nominatim
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent="KapilsGeocoder")
location = locator.geocode("Toronto, Canada")


In [29]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_temp=location
df_temp['Address'] = df_temp['PostalCode'].astype(str) + ',' + ' Toronto'   
df_temp['Location'] = df_temp['Address'].apply(geocode)
df_temp['Point'] = df_temp['Location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_temp[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_temp['Point'].tolist(), index=df_temp.index)



TypeError: tuple indices must be integers or slices, not str

In [ ]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

<b> Explore and cluster the neighborhoods in Toronto </b>

In [ ]:
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer") # Changed user agent due to collision
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))


<b> Generate maps to visualize your neighborhoods and how they cluster together </b>

In [ ]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

In [43]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto